# Immobiliare.it project

## Context


Recently I have been reading few books on the british real estate business model "Buy-to-Let" and I really got fascinated by it. Literally "Buy to let" refers to the purchase of a property specifically to let out, that is to rent it out.

I started wondering whether it could be possible to implement this businees in  different area, considering that the house market in the UK is quite inflated and in a city like London (the one I am currently living in), the price are usually really high. 

Having lived for most of my life in Rome and almost an year in Barcelona, I decided to scrape data about housing market of those two cities, analyse it and understand whther the `buy to let` model can be implemented there (feasibility) and in case possible investment required.

For each city I will try to answer to the following questions:

* Are there areas within the city where the `Buy-to-let` businees can be implemented?

* What would be the capital required for the investment? 

* what will be the forecasted ROI on the investment?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import scraped files csv and concatenate them

To get the info required to start this project, I used ParseHub to scrape relevant data from Immobiliare.it website. 

Immobiliare.it is the leading real estate portal in Italy for both buying and renting.

For this research, I decided to focus mainly on central neighboroughs or at least the ones that are just few kms away from the city center. 

I divided my scraping into 4 price range searches, hence obtaining 4 different .csv files.

In [19]:
# Import csv files into Dataframe 
df1 = pd.read_csv(r'C:\Users\david\OneDrive\Desktop\Python\ParseHub\Projects_CSV\03.immobiliare\Immobiliare_to280000.csv')
df2 = pd.read_csv(r'C:\Users\david\OneDrive\Desktop\Python\ParseHub\Projects_CSV\03.immobiliare\Immobiliare_from281000_to445000.csv')
df3 = pd.read_csv(r'C:\Users\david\OneDrive\Desktop\Python\ParseHub\Projects_CSV\03.immobiliare\Immobiliare_from445000to1000000.csv')
df4 = pd.read_csv(r'C:\Users\david\OneDrive\Desktop\Python\ParseHub\Projects_CSV\03.immobiliare\Immobiliare_over_1Million.csv')


In [20]:
# Concatenate the 4 different files into one dataframe
df = pd.concat([df1, df2, df3, df4])
df.head()

,property_name_name,property_name_url,property_name_price,property_name_rooms,property_name_surface,property_name_bathrooms,property_name_floor,property_name_description,property_name_original_price,property_name_price_reduction
0,Penthouse via di Santa Maria della Speranza 11...,https://www.immobiliare.it/en/annunci/85455828/,€ 235.000,2,52 m2\nsurface,1,4\nfloor,NaN,NaN,NaN
1,"3-room flat via Eugenio Torelli Viollier, Piet...",https://www.immobiliare.it/en/annunci/85452104/,€ 238.000,3,87 m2\nsurface,1,2\nfloor,"Tiburtina, apartment for sale adjacent to Larg...",NaN,NaN
2,"2-room flat via Satrico, San Giovanni, Roma",https://www.immobiliare.it/en/annunci/85396286/,€ 255.000,2,60 m2\nsurface,1,5\nfloor,"S. Giovanni, Two-room apartment for sale on th...",NaN,NaN
3,"2-room flat piazza Giovanni da Triora, Garbate...",https://www.immobiliare.it/en/annunci/85393966/,€ 245.000,2,55 m2\nsurface,1,R\nfloor,NaN,NaN,NaN
4,"Studio via Morlupo 13, Fleming, Roma",https://www.immobiliare.it/en/annunci/85282655/,€ 109.000,1,23 m2\nsurface,1,T\nfloor,NaN,NaN,NaN


In [21]:
# Check columns and their names
df.columns

Index(['property_name_name', 'property_name_url', 'property_name_price',
       'property_name_rooms', 'property_name_surface',
       'property_name_bathrooms', 'property_name_floor',
       'property_name_description', 'property_name_original_price',
       'property_name_price_reduction'],
      dtype='object')

In [22]:
# Rename df columns
df = df.rename(columns={'property_name_name': 'property', 'property_name_url':'url', 'property_name_price':'price',
       'property_name_rooms':'rooms', 'property_name_surface':'surface',
       'property_name_bathrooms':'bathrooms', 'property_name_floor':'floor',
       'property_name_description':'description', 'property_name_original_price':'original_price',
       'property_name_price_reduction':'price_reduction'})
df.head()

,property,url,price,rooms,surface,bathrooms,floor,description,original_price,price_reduction
0,Penthouse via di Santa Maria della Speranza 11...,https://www.immobiliare.it/en/annunci/85455828/,€ 235.000,2,52 m2\nsurface,1,4\nfloor,NaN,NaN,NaN
1,"3-room flat via Eugenio Torelli Viollier, Piet...",https://www.immobiliare.it/en/annunci/85452104/,€ 238.000,3,87 m2\nsurface,1,2\nfloor,"Tiburtina, apartment for sale adjacent to Larg...",NaN,NaN
2,"2-room flat via Satrico, San Giovanni, Roma",https://www.immobiliare.it/en/annunci/85396286/,€ 255.000,2,60 m2\nsurface,1,5\nfloor,"S. Giovanni, Two-room apartment for sale on th...",NaN,NaN
3,"2-room flat piazza Giovanni da Triora, Garbate...",https://www.immobiliare.it/en/annunci/85393966/,€ 245.000,2,55 m2\nsurface,1,R\nfloor,NaN,NaN,NaN
4,"Studio via Morlupo 13, Fleming, Roma",https://www.immobiliare.it/en/annunci/85282655/,€ 109.000,1,23 m2\nsurface,1,T\nfloor,NaN,NaN,NaN


## Data Cleaning

### Area

Checking the property column, I noticed that each entry generally ends with the city name after a comma and before that, between 2 commas, is specified the area where the property is located. Using string's methods I will pull out the `area` information from `property` and create a proper column for it. 

In [23]:
df.property[0]

0    Penthouse via di Santa Maria della Speranza 11...
0     3-room flat via Gaetano Casati, Garbatella, Roma
0    Apartment via Aurelia Antica 200, Gregorio VII...
0                 Penthouse via Germanico, Prati, Roma
Name: property, dtype: object

In [24]:
# Pull out the area of Rome from property
# First remove Rome from the string
df['area'] = df['property'].apply(lambda x: x.split()[:-1])
# Join again together list elements to get a string object
df['area'] = df['area'].apply(lambda x: ' '.join(x))
#  Get the part after the comma
df['area'] = df['area'].apply(lambda x: x.split(',')[1])
# Turn the column dtype to string
df['area'] =df['area'].astype(str)
# Add to n/a all those element that not represent a geagraphic area
df['area'] = df['area'].apply(lambda x: 'nan' if 'sq.m.' in x.lower() else x)
df['area'] = df['area'].apply(lambda x: 'nan' if 'floor' in x.lower() else x)
df['area'] = df['area'].apply(lambda x: 'nan' if ' semi-basement' in x.lower() else x)
df['area'] = df['area'].apply(lambda x: 'nan' if 'condition' in x.lower() else x)
df['area'] = df['area'].apply(lambda x: 'nan' if  'mezzanine'  in x.lower() else x)
df['area'] = df['area'].apply(lambda x: 'nan' if 'multiple levels'  in x.lower() else x)
df['area'] = df['area'].apply(lambda x: 'nan' if 'refurbished' in x.lower() else x)
# Remove street address number 

In [25]:
# Use regex to replace digits with empty string
import re
pattern = r'[0-9]'
# Match all digits in the string and replace them by empty string
df['area'] = df['area'].apply(lambda x: re.sub(pattern, '', x))
df.area.value_counts()

nan                            775
 Talenti - Monte Sacro         516
 Africano - Villa Chigi        468
 Gregorio VII - Piccolomini    419
 Marconi                       413
                              ... 
 -A                              1
                                 1
 -B                              1
 B                               1
 --                              1
Name: area, Length: 106, dtype: int64

In [26]:
# Replace all values that have lenght <= 3 (like -A or -B) with nan 
df['area'] = df['area'].apply(lambda x: 'nan' if len(x) <= 3 else x)
df['area'].value_counts()

nan                            1451
 Talenti - Monte Sacro          516
 Africano - Villa Chigi         468
 Gregorio VII - Piccolomini     419
 Marconi                        413
                               ... 
 Valle Aurelia                   29
 Villaggio Olimpico              28
 Largo Argentina                 20
 Tor di Quinto                    3
 quartiere trieste                1
Name: area, Length: 89, dtype: int64

## Price

In order to work with the `price` column as numerical feature, some manipulation and cleaning duties must be implemented. 

In [27]:
# Let's check some of the values withnin the column
df.price.value_counts()

€ 350.000                      195
€ 299.000                      192
€ 249.000                      184
€ 395.000                      176
€ 295.000                      168
                              ... 
€ 68.000\n€ 88.000(-22.7%)       1
€ 316.500                        1
€ 470.000\n€ 495.000(-5.1%)      1
€ 568.000                        1
€ 170.500                        1
Name: price, Length: 1449, dtype: int64

In [28]:
# Remove the '€' char and the '.' from the column and then convert it to numerical
df['price'] = df['price'].apply(lambda x: x.replace('€', '').replace('.', ''))
df.price.value_counts()

 350000                   195
 299000                   192
 249000                   184
 395000                   176
 295000                   168
                         ... 
 335025                     1
 2359000                    1
 650000\n 695000(-65%)      1
 54000                      1
 251000                     1
Name: price, Length: 1449, dtype: int64

In [29]:
# Some entries have an extra part which has to be removed
df['price'] = df['price'].apply(lambda x: x.split()[0])

In [30]:
# Check again the price column values
df.price.value_counts()

299000     199
350000     199
249000     189
395000     178
295000     174
          ... 
111375       1
4400000      1
1524000      1
2690000      1
3190000      1
Name: price, Length: 1122, dtype: int64

In [31]:
# Turn the column to numerical
df['price'] = df['price'].astype(float)
df.dtypes

property            object
url                 object
price              float64
rooms               object
surface             object
bathrooms           object
floor               object
description         object
original_price      object
price_reduction     object
area                object
dtype: object

### Rooms

Having a first glance to the `rooms` column, I noticed that some special characters '+' and unrealistic values (i.e. 1.580) are included.

In [32]:
# Let's have a quick look to the column
df.rooms.value_counts()

3      4991
2      3588
4      3239
5+     1967
5      1661
       ... 
84        1
76        1
71        1
106       1
32        1
Name: rooms, Length: 70, dtype: int64

In [33]:
# Remove the special char '+'
df['rooms'] = df['rooms'].apply(lambda x: x.replace('+', ''))
# Convert the feature to numerical
df['rooms'] = df['rooms'].astype(float)
# Replace values greater than 5 with 'NaN' 
df['rooms'] = df['rooms'].apply(lambda x: 'NaN' if x > 5  else x)
# Create a dummy feature to save the missing values
df['room_is_missing'] = df['rooms'].apply(lambda x: 1 if x == 'NaN' else 0)

In [34]:
# Convert again the column to string and then replace the 'NaN' with the numerical nan
df['rooms'] = df['rooms'].astype(str)
df= df.replace('NaN', np.nan)
df.rooms.value_counts()

3.0     4991
5.0     3628
2.0     3588
4.0     3239
1.0      647
1.58       1
1.15       1
1.22       1
1.3        1
2.5        1
2.4        1
Name: rooms, dtype: int64

In [36]:
# Check how many nan values we have within the column
df.rooms.isna().sum()

127

In [64]:
# Checking the dataframe, there are still 127 nan values. 
# First fill the nan values with column median
df['rooms'] = df['rooms'].fillna(df.rooms.median())
# Finally convert the column to numerical
df['rooms'] = df['rooms'].astype(float)
df.rooms.value_counts()

3.00    5118
5.00    3628
2.00    3588
4.00    3239
1.00     647
2.40       1
1.15       1
1.30       1
1.58       1
1.22       1
2.50       1
Name: rooms, dtype: int64

### Surface

In [66]:
# Let's check first entries for Surface cdf.isna().sum()olumn
df.surface.value_counts()

100 m2\nsurface      602
90 m2\nsurface       585
80 m2\nsurface       525
120 m2\nsurface      516
70 m2\nsurface       503
                    ... 
6\nfloor               1
469 m2\nsurface        1
311 m2\nsurface        1
1.550 m2\nsurface      1
266 m2\nsurface        1
Name: surface, Length: 459, dtype: int64

In [67]:
# Extract just the number from the string
# First convert all values to string dtype
df['surface'] = df['surface'].astype(str)
# Split the string and pull out first element
df['surface'] = df['surface'].apply(lambda x: x.split()[0])
df.surface.value_counts()

100      602
90       585
80       525
120      516
70       503
        ... 
460        1
535        1
2.740      1
445        1
1.317      1
Name: surface, Length: 451, dtype: int64

In [68]:
# Removing not numerical entries
df['surface'] = df['surface'].str.extract('(\d+)')
# Convert again the column to float
df['surface'] = df['surface'].astype(float)

In [69]:
df.surface.value_counts()

100.0    602
90.0     585
80.0     525
120.0    516
70.0     503
        ... 
262.0      1
343.0      1
405.0      1
258.0      1
327.0      1
Name: surface, Length: 423, dtype: int64

###  Bathrooms

In [70]:
# Check values within the Bathrooms column
df['bathrooms'].value_counts()

1             7640
2             5791
3             1463
3+             839
4               77
T               68
5               57
6               27
S               25
7               17
R               14
8                7
guaranteed       5
A                3
9                1
10               1
Name: bathrooms, dtype: int64

In [71]:
# 
df['bathrooms'] = df['bathrooms'].astype(str)
df['bathrooms'] = df['bathrooms'].apply(lambda x: x.replace('+','').replace('T', '').replace('S', '').replace('R', '').replace('guaranteed', '').replace('A', '').replace('nan', ''))
df['bathrooms'] = df['bathrooms'].apply(lambda x: 'NaN' if x == '' else x)
# Create dummy column to record missing values
df['bathroom_is_missing'] = df['bathrooms'].apply(lambda x: 1 if x == 'NaN' else 0)
df= df.replace('NaN', np.nan)
df['bathrooms'] = df['bathrooms'].fillna(df.bathrooms.median())
df.bathrooms.value_counts()

1      7640
2      5791
3      2302
2.0     306
4        77
5        57
6        27
7        17
8         7
9         1
10        1
Name: bathrooms, dtype: int64

In [72]:
df['bathrooms'] = df['bathrooms'].astype(int)

In [73]:
df['bathrooms'].value_counts()

1     7640
2     6097
3     2302
4       77
5       57
6       27
7       17
8        7
10       1
9        1
Name: bathrooms, dtype: int64

### Floor

From the Immobiliare website I got the following acronym translation:

* T: Ground floor
* R: Mezzanine floor
* S: Semi-Basement
* A: Penthouse

In [74]:
df[df['floor'] == 'A'] ['url']

Series([], Name: url, dtype: object)

In [75]:
df.floor.iloc[0].split()

['4', 'floor']

In [76]:
df['floor'] = df['floor'].astype(str)
df['floor'] = df['floor'].apply(lambda x: x.split()[0])
df.floor.value_counts()

1           2920
2           2660
3           2103
T           1814
4           1648
nan         1440
5           1034
R            559
6            550
S            430
7            364
property     332
8            196
A             90
9             72
10            10
11+            4
Name: floor, dtype: int64

In [77]:
df['floor'] = df['floor'].apply(lambda x: x.replace('nan', 'NaN').replace('property', 'NaN').replace('T', 'Ground').replace('R', 'Mezzanine').replace('S', 'Semi-basement').replace('A', 'Penthouse'))
df['floor'].value_counts()

1                2920
2                2660
3                2103
Ground           1814
NaN              1772
4                1648
5                1034
Mezzanine         559
6                 550
Semi-basement     430
7                 364
8                 196
Penthouse          90
9                  72
10                 10
11+                 4
Name: floor, dtype: int64

In [78]:
df.head()

,property,url,price,rooms,surface,bathrooms,floor,description,original_price,price_reduction,area,room_is_missing,bathroom_is_missing
0,Penthouse via di Santa Maria della Speranza 11...,https://www.immobiliare.it/en/annunci/85455828/,235000.0,2.0,52.0,1,4,NaN,NaN,NaN,Nuovo Salario,0,0
1,"3-room flat via Eugenio Torelli Viollier, Piet...",https://www.immobiliare.it/en/annunci/85452104/,238000.0,3.0,87.0,1,2,"Tiburtina, apartment for sale adjacent to Larg...",NaN,NaN,Pietralata,0,0
2,"2-room flat via Satrico, San Giovanni, Roma",https://www.immobiliare.it/en/annunci/85396286/,255000.0,2.0,60.0,1,5,"S. Giovanni, Two-room apartment for sale on th...",NaN,NaN,San Giovanni,0,0
3,"2-room flat piazza Giovanni da Triora, Garbate...",https://www.immobiliare.it/en/annunci/85393966/,245000.0,2.0,55.0,1,Mezzanine,NaN,NaN,NaN,Garbatella,0,0
4,"Studio via Morlupo 13, Fleming, Roma",https://www.immobiliare.it/en/annunci/85282655/,109000.0,1.0,23.0,1,Ground,NaN,NaN,NaN,Fleming,0,0


### Original Price

In [79]:
df.original_price.value_counts()

€ 595.000    11
€ 195.000     9
€ 249.000     7
€ 295.000     7
€ 400.000     6
             ..
€ 249.075     1
€ 390.000     1
€ 490.000     1
€ 316.000     1
€ 445.000     1
Name: original_price, Length: 191, dtype: int64

In [80]:
# Turn column to string and then remove extra characters
df['original_price'] = df['original_price'].astype(str)
df['original_price'] = df['original_price'].apply(lambda x: x.replace('€', '').replace('.', ''))
df.original_price.value_counts()

nan         15822
 595000        11
 195000         9
 295000         7
 249000         7
            ...  
 88000          1
 305000         1
 1080000        1
 2800000        1
 849000         1
Name: original_price, Length: 192, dtype: int64

In [81]:
# Most values are equal to nan, which means that there was no price variation. Therefore we will loop trough the column to set the nan  value equal to df.price values
for item in df['original_price']:
    if item == 'nan':
        df['original_price'] = df['price']
        
df.original_price.value_counts()

299000.0     199
350000.0     199
249000.0     189
395000.0     178
295000.0     174
            ... 
142000.0       1
1165000.0      1
341326.0       1
43500.0        1
1598000.0      1
Name: original_price, Length: 1122, dtype: int64

In [82]:
df['original_price'] = df['original_price'].astype(float)

### Price reduction

In [83]:
df.price_reduction.value_counts()

(-5.4%)     21
(-6.3%)     16
(-5.1%)     16
(-5.9%)     14
(-5.8%)     13
            ..
(-13.3%)     1
(-20.2%)     1
(-19.5%)     1
(-15.6%)     1
(-13.1%)     1
Name: price_reduction, Length: 102, dtype: int64

In [84]:
df = df.fillna(value=0)
df.price_reduction.value_counts()

0           15822
(-5.4%)        21
(-6.3%)        16
(-5.1%)        16
(-5.9%)        14
            ...  
(-13.3%)        1
(-20.2%)        1
(-19.5%)        1
(-15.6%)        1
(-13.1%)        1
Name: price_reduction, Length: 103, dtype: int64

In [85]:
# Remove all special characters from the column
df['price_reduction'] = df['price_reduction'].astype(str)
df['price_reduction'] = df['price_reduction'].apply(lambda x: x.replace('(', '').replace('-', '').replace('%', '').replace(')', ''))
df['price_reduction'] = df['price_reduction'].astype(float)
df.price_reduction.value_counts()

0.0     15822
5.4        21
6.3        16
5.1        16
5.9        14
        ...  
10.4        1
18.2        1
15.5        1
13.6        1
14.6        1
Name: price_reduction, Length: 103, dtype: int64

In [86]:
df.head()

,property,url,price,rooms,surface,bathrooms,floor,description,original_price,price_reduction,area,room_is_missing,bathroom_is_missing
0,Penthouse via di Santa Maria della Speranza 11...,https://www.immobiliare.it/en/annunci/85455828/,235000.0,2.0,52.0,1,4,0,235000.0,0.0,Nuovo Salario,0,0
1,"3-room flat via Eugenio Torelli Viollier, Piet...",https://www.immobiliare.it/en/annunci/85452104/,238000.0,3.0,87.0,1,2,"Tiburtina, apartment for sale adjacent to Larg...",238000.0,0.0,Pietralata,0,0
2,"2-room flat via Satrico, San Giovanni, Roma",https://www.immobiliare.it/en/annunci/85396286/,255000.0,2.0,60.0,1,5,"S. Giovanni, Two-room apartment for sale on th...",255000.0,0.0,San Giovanni,0,0
3,"2-room flat piazza Giovanni da Triora, Garbate...",https://www.immobiliare.it/en/annunci/85393966/,245000.0,2.0,55.0,1,Mezzanine,0,245000.0,0.0,Garbatella,0,0
4,"Studio via Morlupo 13, Fleming, Roma",https://www.immobiliare.it/en/annunci/85282655/,109000.0,1.0,23.0,1,Ground,0,109000.0,0.0,Fleming,0,0


In [87]:
df.dtypes

property                object
url                     object
price                  float64
rooms                  float64
surface                float64
bathrooms                int32
floor                   object
description             object
original_price         float64
price_reduction        float64
area                    object
room_is_missing          int64
bathroom_is_missing      int64
dtype: object

In [110]:
# Replace not nan values with a string
df['description'] = df['description'].apply(lambda x: 'Description not available' if x ==0 else x)
df['description'] = df['description'].astype(str)
df.description.value_counts()

Description not available                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               